In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time
import random

# Headers ekleyelim (User-Agent)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Excel dosyasını oku
try:
    df = pd.read_excel('ayakkabıürunler.xlsx')
    urls = df['URLs'].tolist()  # URLs sütununu bir listeye dönüştür
except FileNotFoundError:
    print("Hata: Excel dosyası bulunamadı.")
    exit()

# Her bir URL için işlem yap
for url in urls:
    url = 'https:' + url  # URL'ye 'https' şeması ekle
    print(80*"-")
    print("Product URL:", url)
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Hata:", e)
        continue
    
    if response.status_code == 200:
        # BeautifulSoup kullanarak HTML içeriğini analiz et
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tüm <script> etiketlerini bul
        scripts = soup.find_all('script')

        # Bilgileri saklamak için bir sözlük oluştur
        data = {}
        data['product_url'] = url  # Ürün URL'sini ekle

        # Her bir <script> etiketinin içeriğini kontrol et
        for script in scripts:
            # Eğer içerikte "window.runParams" varsa
            if 'window.runParams' in script.text:
                # Script içeriğini yazdır
                run_params_content = script.text
                
                # imagePathList'den fotoğraf URL'lerini al
                image_urls_match = re.search(r'"imagePathList":(\[.*?\])', run_params_content)
                if image_urls_match:
                    image_urls_str = image_urls_match.group(1)
                    image_urls = json.loads(image_urls_str)
                    # Maksimum 4 URL al
                    image_urls = image_urls[:4]
                    data['image_urls'] = image_urls 
                    # URL'leri yazdır
                    for img_url in image_urls:
                        print("Image URL:", img_url)
                
                # "formatTradeCount" değerini bul
                format_trade_count_match = re.search(r'"formatTradeCount":"(.*?)"', run_params_content)
                if format_trade_count_match:
                    data['format_trade_count'] = format_trade_count_match.group(1)
                    print("Count:", data['format_trade_count'])
                
                # "evarageStar" değerini bul
                evarage_star_match = re.search(r'"evarageStar":"(.*?)"', run_params_content)
                if evarage_star_match:
                    data['evarage_star'] = evarage_star_match.group(1)
                    print("Average Star:", data['evarage_star'])

                # "keywords" değerini bul
                keywords_match = re.search(r'"keywords":"(.*?)"', run_params_content)
                if keywords_match:
                    data['keywords'] = keywords_match.group(1)
                    print("Keywords:", data['keywords'])
                    
                # "ogTitle" değerini bul
                og_title_match = re.search(r'"ogTitle":"(.*?)"', run_params_content)
                if og_title_match:
                    data['og_title'] = og_title_match.group(1)
                    title_parts = data['og_title'].split('|')
                    if len(title_parts) > 1:
                        print("Title:", title_parts[1].strip())

                # "description" değerini bul
                description_match = re.search(r'"description":"(.*?)"', run_params_content)
                if description_match:
                    data['description'] = description_match.group(1)
                    print("Description:", data['description'])
                
                # Fiyat bilgilerini çek
                price_info = re.search(r'"skuAmount":\{"currency":"(.*?)","formatedAmount":"(.*?)","value":(.*?)\}', run_params_content)
                if price_info:
                    data['currency'] = price_info.group(1)
                    data['formated_price'] = price_info.group(2)
                    data['price'] = price_info.group(3)
                    print("Currency:", data['currency'])
                    print("Formatted Price:", data['formated_price'])
                    print("Price:", data['price'])

                discount_info = re.search(r'"skuActivityAmount":\{"currency":"(.*?)","formatedAmount":"(.*?)","value":(.*?)\}', run_params_content)
                if discount_info:
                    data['discount_currency'] = discount_info.group(1)
                    data['discount_formated_price'] = discount_info.group(2)
                    data['discount_price'] = discount_info.group(3)
                    print("Discount Currency:", data['discount_currency'])
                    print("Discount Formatted Price:", data['discount_formated_price'])
                    print("Discount Price:", data['discount_price'])
                
                discount_rate_match = re.search(r'"discount":"(.*?)"', run_params_content)
                if discount_rate_match:
                    data['discount_rate'] = discount_rate_match.group(1)
                    print("Discount Rate:", data['discount_rate'])
                    
                # "packageComponent" değerlerini bul
                package_component_match = re.search(r'"packageComponent":\{(.*?)\}', run_params_content)
                if package_component_match:
                    package_component_content = package_component_match.group(1)
                    # Virgülle ayrılan JSON değerlerini tek tek işleyerek bir dict'e dönüştürelim
                    package_data = {}
                    for item in package_component_content.split(','):
                        key, value = item.split(':')
                        key = key.strip('"')
                        try:
                            value = json.loads(value)
                        except json.JSONDecodeError:
                            value = value.strip('"')
                        package_data[key] = value
                    data['package_component'] = package_data
                    for key, value in package_data.items():
                        print(f"{key}: {value}")
                
                # Eğer içerikte "productPropComponent" varsa
                if 'productPropComponent' in script.text:
                    # Script içeriğini yazdır
                    prop_component_content = script.text

                    # "attrValue" ve "propertyValueName" değerlerini bul
                    attr_value_matches = re.findall(r'"attrValue":"(.*?)"', prop_component_content)
                    property_value_name_matches = re.findall(r'"attrValue":"(.*?)".*?"attrName":"(.*?)"', prop_component_content)

                    # Her bir eşleşmeyi yazdır ve sakla
                    attributes = {}
                    for attr_name, attr_value in property_value_name_matches:
                        attributes[attr_value] = attr_name
                        print(attr_value + ":", attr_name)
                    data['attributes'] = attributes

                # Eğer içerikte "skuPropertyValues" varsa
                if 'skuPropertyValues' in script.text:
                    # Script içeriğini yazdır
                    prop_component_content = script.text

                    # Renk ve Boyut bilgilerini bul
                    color_sizes_data = re.findall(r'"skuPropertyValues":\[(.*?)\]', prop_component_content)

                    colors = []
                    sizes = []

                    for match in color_sizes_data:
                        # Her bir match'i temizleyip JSON formatına uygun hale getirelim
                        cleaned_match = re.sub(r'\\', '', match)
                        try:
                            properties = json.loads(f'[{cleaned_match}]')
                        except json.JSONDecodeError:
                            continue
                        for prop in properties:
                            if 'skuColorValue' in prop:
                                color_name = prop.get('propertyValueName', 'N/A')
                                colors.append(color_name)
                            if 'propertyValueDisplayName' in prop:
                                size_name = prop.get('skuPropertyTips', 'N/A')
                                sizes.append(size_name)

                    data['colors'] = colors
                    data['sizes'] = sizes

                    for color in colors:
                        for size in sizes:
                            print(f"Renk: {color}")
                            print(f"Beden: {size}")
                            if 'discount_formated_price' in data:
                                print(f"Fiyat: {data['discount_formated_price']} (İndirimli)")
                            else:
                                print(f"Fiyat: {data['formated_price']}")

                break  # İlk eşleşmeyi bulduktan sonra döngüden çık

        # Verileri JSON olarak kaydet
        with open('ayakkabıürunler.json', 'a', encoding='utf-8') as f:  # 'a' append mode to avoid overwriting
            json.dump(data, f, ensure_ascii=False, indent=4)
            f.write(',\n')  # JSON objects separated by a comma for multiple entries
    else:
        print("Hata: Sayfa yüklenemedi")

    # Her ürün isteği arasında rastgele bir süre bekle (3 ile 7 saniye arasında)
    sleep_time = random.uniform(3, 7)
    print(f"Bekleme süresi: {sleep_time:.2f} saniye")
    time.sleep(sleep_time)


--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005006485865233.html?spm=a2g0o.categorymp.prodcutlist.1.76681yR51yR5mT&pdp_ext_f=%7B%22sku_id%22%3A%2212000038389529187%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Image URL: https://ae01.alicdn.com/kf/Sc154b46e9ec24c77b9cae051d227b57bE/FitVille-erkek-diyabetik-ayakkab-deri-ortopedik-ayak-parmaklar-i-mi-ayaklar-i-in-ekstra-geni-y.jpg
Image URL: https://ae01.alicdn.com/kf/Sc189e071df3540b8a2c6aa6e2f825476q/FitVille-erkek-diyabetik-ayakkab-deri-ortopedik-ayak-parmaklar-i-mi-ayaklar-i-in-ekstra-geni-y.jpg
Image URL: https://ae01.alicdn.com/kf/S0934690db5ad4b57b2ec972571903b7d0/FitVille-erkek-diyabetik-ayakkab-deri-ortopedik-ayak-parmaklar-i-mi-ayaklar-i-in-ekstra-geni-y.jpg
Image URL: https://ae01.alicdn.com/kf/Sfa1ea162137f415c82156ebf177907d5N/FitVille-erkek-diyabetik-ayakkab-deri-ortopedik-ayak-parmaklar-i-mi-ayaklar-i-in-ekstra-geni-y.jpg
Bekleme süresi: 5

--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005006196702847.html?spm=a2g0o.categorymp.prodcutlist.3.76681yR51yR5mT&pdp_ext_f=%7B%22sku_id%22%3A%2212000036225283635%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Image URL: https://ae01.alicdn.com/kf/Sdc9f24a7574c4aa1a8e1372eef79827f8/2024-yeni-stil-yar-m-Slip-on-ayakkab-lar-rahat-d-z-ayakkab-Mesh-erkek-ayakkab.jpg
Image URL: https://ae01.alicdn.com/kf/Sa4c1518bfddd46308c5ee991e1518fddT/2024-yeni-stil-yar-m-Slip-on-ayakkab-lar-rahat-d-z-ayakkab-Mesh-erkek-ayakkab.jpg
Image URL: https://ae01.alicdn.com/kf/S23fe312070474161832264b2ce3a21859/2024-yeni-stil-yar-m-Slip-on-ayakkab-lar-rahat-d-z-ayakkab-Mesh-erkek-ayakkab.jpg
Image URL: https://ae01.alicdn.com/kf/S04d64b8148084e9292f5b29bce3a3ef3C/2024-yeni-stil-yar-m-Slip-on-ayakkab-lar-rahat-d-z-ayakkab-Mesh-erkek-ayakkab.jpg
Count: 474
Average Star: 4.3
Keywords: Aliexpress, Aliexpress.com, Online alışveriş, 

--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005005083785174.html?spm=a2g0o.categorymp.prodcutlist.6.76681yR51yR5mT&pdp_ext_f=%7B%22sku_id%22%3A%2212000031604446482%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Image URL: https://ae01.alicdn.com/kf/S74c6c9aea0f8469eac28f317181862f2N/ZZFABER-yal-nayak-Sneakers-erkekler-kad-nlar-i-in-yumu-ak-rahat-ayakkab-lar-rahat-nefes.jpg
Image URL: https://ae01.alicdn.com/kf/S4054ab3ab2cb42fb8882491b1cc65952u/ZZFABER-yal-nayak-Sneakers-erkekler-kad-nlar-i-in-yumu-ak-rahat-ayakkab-lar-rahat-nefes.jpg
Image URL: https://ae01.alicdn.com/kf/S8281c4c303b94387a93d8ed0d68c5794d/ZZFABER-yal-nayak-Sneakers-erkekler-kad-nlar-i-in-yumu-ak-rahat-ayakkab-lar-rahat-nefes.jpg
Image URL: https://ae01.alicdn.com/kf/Sd33e12536f094c488049c6bae9ac3c63r/ZZFABER-yal-nayak-Sneakers-erkekler-kad-nlar-i-in-yumu-ak-rahat-ayakkab-lar-rahat-nefes.jpg
Count: 500+
Average Star: 4.9
Keywords: Aliex

--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005006472433146.html?spm=a2g0o.categorymp.prodcutlist.8.76681yR51yR5mT&pdp_ext_f=%7B%22sku_id%22%3A%2212000037321342187%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Image URL: https://ae01.alicdn.com/kf/Seb39ab0179844a66aa22a08e3f277cb4k/2024-yeni-erkekler-masaj-terlikleri-slaytlar-kapal-a-k-sandalet-plaj-rahat-ayakkab-lar-rahat-taban.jpg
Image URL: https://ae01.alicdn.com/kf/Se1a2b711bb9b44e0bd93581e48348907N/2024-yeni-erkekler-masaj-terlikleri-slaytlar-kapal-a-k-sandalet-plaj-rahat-ayakkab-lar-rahat-taban.jpg
Image URL: https://ae01.alicdn.com/kf/S6ba7a6f8ec234f66a7840bce239e91a8b/2024-yeni-erkekler-masaj-terlikleri-slaytlar-kapal-a-k-sandalet-plaj-rahat-ayakkab-lar-rahat-taban.jpg
Image URL: https://ae01.alicdn.com/kf/Sa7822e18060e4c60963f96b7272d42ebF/2024-yeni-erkekler-masaj-terlikleri-slaytlar-kapal-a-k-sandalet-plaj-rahat-ayakkab-lar-rahat-taban.jpg
C

--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/4000383125948.html?spm=a2g0o.categorymp.prodcutlist.11.76681yR51yR5mT&pdp_ext_f=%7B%22sku_id%22%3A%2210000001565823495%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Image URL: https://ae01.alicdn.com/kf/U4830b97ad1834c5380a3583e32cceaabi/Punk-kad-nlar-Sneakers-kaya-6cm-Platform-ayakkab-lar-kad-n-Creepers-kad-n-rahat-daireler.jpg
Image URL: https://ae01.alicdn.com/kf/S031066c6a2ea474890ceb13b923e7450Y/Punk-kad-nlar-Sneakers-kaya-6cm-Platform-ayakkab-lar-kad-n-Creepers-kad-n-rahat-daireler.jpg
Image URL: https://ae01.alicdn.com/kf/U4de0a433a7f7444fb0f1c7c7d9003bfaK/Punk-kad-nlar-Sneakers-kaya-6cm-Platform-ayakkab-lar-kad-n-Creepers-kad-n-rahat-daireler.jpg
Image URL: https://ae01.alicdn.com/kf/S41bd91dd69594e8784078321dc1a6913y/Punk-kad-nlar-Sneakers-kaya-6cm-Platform-ayakkab-lar-kad-n-Creepers-kad-n-rahat-daireler.jpg
Count: 428
Average Star: 4.6
Keywords: Kadı

...

--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005005016033414.html?spm=a2g0o.categorymp.prodcutlist.78.76681yR51yR5mT&pdp_ext_f=%7B%22sku_id%22%3A%2212000031350816395%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Image URL: https://ae01.alicdn.com/kf/S248bbf42292e4a9496e236bc5d419694I/Ngouxm-erkekler-ef-ayakkab-kad-nlar-kaymaz-su-ge-irmez-ya-ge-irmez-mutfak-hem-ire.jpg
Image URL: https://ae01.alicdn.com/kf/S28f5845cdd134dd3a3abe450ccfec413y/Ngouxm-erkekler-ef-ayakkab-kad-nlar-kaymaz-su-ge-irmez-ya-ge-irmez-mutfak-hem-ire.jpg
Image URL: https://ae01.alicdn.com/kf/Scf3be4d8d8cb42908fdc924ec82e44aer/Ngouxm-erkekler-ef-ayakkab-kad-nlar-kaymaz-su-ge-irmez-ya-ge-irmez-mutfak-hem-ire.jpg
Image URL: https://ae01.alicdn.com/kf/Se9e145b8bcbe400081cf4f01669f53a6b/Ngouxm-erkekler-ef-ayakkab-kad-nlar-kaymaz-su-ge-irmez-ya-ge-irmez-mutfak-hem-ire.jpg
Count: 324
Average Star: 4.6
Keywords: Aliexpress, Aliexpress.com, O

Image URL: https://ae01.alicdn.com/kf/S201d964d5e3d4de595bad983e27c1b3cg/Bale-daireler-kad-n-ayakkab-43-Mary-Jane-ayakkab-yuvarlak-ayak-kad-n-ayakkab-yay-ipek.jpg
Image URL: https://ae01.alicdn.com/kf/S07d4475fd8304010a3bb4dc5d160b196z/Bale-daireler-kad-n-ayakkab-43-Mary-Jane-ayakkab-yuvarlak-ayak-kad-n-ayakkab-yay-ipek.jpg
Image URL: https://ae01.alicdn.com/kf/S043b0b8f324446819c2710e1ff2ecf45V/Bale-daireler-kad-n-ayakkab-43-Mary-Jane-ayakkab-yuvarlak-ayak-kad-n-ayakkab-yay-ipek.jpg
Image URL: https://ae01.alicdn.com/kf/Sdb0c99eaf9a4402bbbb6ab59808fc831s/Bale-daireler-kad-n-ayakkab-43-Mary-Jane-ayakkab-yuvarlak-ayak-kad-n-ayakkab-yay-ipek.jpg
Count: 500+
Average Star: 4.8
Keywords: Aliexpress, Aliexpress.com, Online alışveriş, Otomotiv, Telefon ve Aksesuarları, Bilgisayarlar ve Elektronik, Moda, Güzellik ve Sağlık, Ev ve Bahçe, Oyuncak ve Spor, Düğün ve Etkinlikler
Title: Bale daireler kadın ayakkabı 43 Mary Jane ayakkabı yuvarlak ayak kadın ayakkabı yay ipek saten daireler kadın ayak

--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005006149542372.html?spm=a2g0o.categorymp.prodcutlist.82.76681yR51yR5mT&pdp_ext_f=%7B%22sku_id%22%3A%2212000035984095912%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Image URL: https://ae01.alicdn.com/kf/Sf430dc8b5c154a56aac38e9786e32525s/Kad-nlar-i-in-ayakkab-2023-s-cak-sat-temel-kad-n-terlik-nefes-rahat-terlik.jpg
Image URL: https://ae01.alicdn.com/kf/S60590e7aa24041cdaa46f2032f0b5bd41/Kad-nlar-i-in-ayakkab-2023-s-cak-sat-temel-kad-n-terlik-nefes-rahat-terlik.jpg
Image URL: https://ae01.alicdn.com/kf/Sda6fc6aa112f4898a5cbb278a869e9c5G/Kad-nlar-i-in-ayakkab-2023-s-cak-sat-temel-kad-n-terlik-nefes-rahat-terlik.jpg
Image URL: https://ae01.alicdn.com/kf/S204caa4855934bf69706ab5764e7b2cds/Kad-nlar-i-in-ayakkab-2023-s-cak-sat-temel-kad-n-terlik-nefes-rahat-terlik.jpg
Count: 254
Average Star: 4.3
Keywords: Aliexpress, Aliexpress.com, Online alışveriş, Otomotiv, T

--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005005928891021.html?spm=a2g0o.categorymp.prodcutlist.85.76681yR51yR5mT&pdp_ext_f=%7B%22sku_id%22%3A%2212000034889877415%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Image URL: https://ae01.alicdn.com/kf/S12f2ae5c20eb435fb96c24bba796ba58M/Punk-a-r-Metal-deri-kad-n-2023-gotik-koyu-dantel-Up-Muffin-Hipster-per-in.jpg
Image URL: https://ae01.alicdn.com/kf/S41bc8f144eea48d69fd67c41eb4a60e83/Punk-a-r-Metal-deri-kad-n-2023-gotik-koyu-dantel-Up-Muffin-Hipster-per-in.jpg
Image URL: https://ae01.alicdn.com/kf/S930c3f733cab4889a2bd4a6170f6911cX/Punk-a-r-Metal-deri-kad-n-2023-gotik-koyu-dantel-Up-Muffin-Hipster-per-in.jpg
Image URL: https://ae01.alicdn.com/kf/S4344ec8ea5564f208ee771f7e9a19673g/Punk-a-r-Metal-deri-kad-n-2023-gotik-koyu-dantel-Up-Muffin-Hipster-per-in.jpg
Count: 84
Average Star: 4.9
Keywords: Aliexpress, Aliexpress.com, Online alışveriş, Otomotiv, Telefo

KeyboardInterrupt: 